In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All"
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import pandas as pd
from pandas import json_normalize

# Establishing connection with Azure

In [ ]:
# Install unixODBC
!apt-get update
!apt-get install -y unixodbc unixodbc-dev

# Install pyodbc
!pip install pyodbc

# Install sqlalchemy
!pip install sqlalchemy

# Install ODBC Driver 17 for SQL Server
!curl https://packages.microsoft.com/keys/microsoft.asc | apt-key add -
!curl https://packages.microsoft.com/config/ubuntu/$(lsb_release -rs)/prod.list | tee /etc/apt/sources.list.d/mssql-release.list
!apt-get update
!ACCEPT_EULA=Y apt-get install -y msodbcsql17

Get:1 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease [1,581 B]
Get:2 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,626 B]
Get:3 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  Packages [1,107 kB]
Get:4 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ Packages [59.5 kB]
Get:5 http://security.ubuntu.com/ubuntu jammy-security InRelease [129 kB]
Hit:6 http://archive.ubuntu.com/ubuntu jammy InRelease
Get:7 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [128 kB]
Get:8 https://r2u.stat.illinois.edu/ubuntu jammy InRelease [6,555 B]
Hit:9 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Get:10 https://r2u.stat.illinois.edu/ubuntu jammy/main amd64 Packages [2,605 kB]
Hit:11 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Get:12 http://security.ubuntu.com/ubuntu jammy-security/universe amd64 Packages [1,162 kB]
Get:13 http://archive.ub

In [ ]:
from sqlalchemy import create_engine
import sqlalchemy


# Define connection details
username = 'serveradmin'  # Replace with your Azure SQL admin username
password = 'Msba2024'        # Replace with your Azure SQL password
server = 'shambhavi64.database.windows.net'
database = 'MiniProject1'

# Connection string for Azure SQL Server
connection_string = f"mssql+pyodbc://{username}:{password}@{server}:1433/{database}?driver=ODBC+Driver+17+for+SQL+Server"
# connection_string = f"mssql+pyodbc://{username}:{password}@{server}:1433/{database}?driver=ODBC+Driver+17+for+SQL+Server;Connection Timeout=60"

engine = create_engine(connection_string)

In [ ]:
#Testing the connection with database
from sqlalchemy import text
with engine.connect() as connection:
    result = connection.execute(text("SELECT GETDATE();"))
    for row in result:
        print("Connected! Server date and time:", row[0])

Connected! Server date and time: 2024-11-05 02:30:10.083000


In [ ]:
from sqlalchemy import Column, Integer, String, Table, MetaData
from sqlalchemy.types import Integer, String

# Define metadata and table with primary key constraint
metadata = MetaData()
check_in_table = Table(
    'check_in', metadata,
    Column('id', Integer, primary_key=True, autoincrement=True),  # Primary key column
    Column('business_id', String(22)),  # Assuming business_id is VARCHAR(22)
    Column('date', String),  # Define other columns as needed
)

# Create the table in the database
metadata.create_all(engine)

# Upload data without creating a new table structure
checkin_df.to_sql(name="check_in", con=engine, if_exists='append', index=False, chunksize=10000)


# Business Queries

In [ ]:
table_name = "business"  # Replace with the table name you want to verify
query = f"SELECT TOP 10 * FROM {table_name};"

# Execute the query and fetch results in a DataFrame
try:
    with engine.connect() as connection:
        # Execute the query and load the result into a DataFrame
        result_df = pd.read_sql_query(text(query), con=connection)
        print(result_df)
except Exception as e:
    print(f"An error occurred: {e}")

              business_id                      name  \
0  Pns2l4eNsfO8kk83dixA6A  Abby Rappoport, LAC, CMQ   
1  mpf3x-BjTdTEA3yCZrAYPw             The UPS Store   
2  tUFrWirKiKi_TAnsVWINQQ                    Target   
3  MTSW4McQd7CbVtyjqoe9mw        St Honore Pastries   
4  mWMc6_wTdE0EUBKIGXDVfA  Perkiomen Valley Brewery   
5  CF33F8-E6oudUQ46HnavjQ            Sonic Drive-In   
6  n_0UpQx1hsNbnPUSlodU8w           Famous Footwear   
7  qkRM_2X51Yqxk3btlwAQIg            Temple Beth-El   
8  k0hlBqXX-Bt0vf1op7Jr1w     Tsevi's Pub And Grill   
9  bBDDEgkFA1Otx9Lfe7BZUQ            Sonic Drive-In   

                                      address            city state  \
0                      1616 Chapala St, Ste 2   Santa Barbara    CA   
1             87 Grasso Plaza Shopping Center          Affton    MO   
2                        5255 E Broadway Blvd          Tucson    AZ   
3                                 935 Race St    Philadelphia    PA   
4                               101 Wal

In [ ]:
#1. finding the city
table_name = "business"  # Replace with the table name you want to verify
query = f"SELECT TOP 20 *, COUNT(*) OVER (PARTITION BY city) AS high_rated_restaurant_count FROM {table_name} WHERE stars >= 3.5  AND [state] = 'IL'  AND categories LIKE '%Restaurants%' AND is_open = 1 ORDER BY high_rated_restaurant_count DESC ;"

# Execute the query and fetch results in a DataFrame
try:
    with engine.connect() as connection:
        # Execute the query and load the result into a DataFrame
        result_df = pd.read_sql_query(text(query), con=connection)
        print(result_df)
except Exception as e:
    print(f"An error occurred: {e}")



               business_id                             name  \
0   RQPDg_bnyjPrk1xiWf3lpA                Uncle Nick's Deli   
1   2UtP4mly2LFZpXtqiwb24g                   The Weingarten   
2   Gt6NclvT89Je0_VHI1eNrQ                Sala Thai Cuisine   
3   omKVwCFLQYs_ySD1bPk6iw   Eckert's Country Store & Farms   
4   uT33RCQUJioJnFEh1C1u7g                 Main Street Cafe   
5   P9kDKqqjdFjjpWIAaxNyRg           Papa Vito's Belleville   
6   S5SYHcyBG9HuaprVO_6jFQ               Sip & Chew Express   
7   -VvhvE-Qs0kjoGidq7XYDQ               Cured And Cultured   
8   fVBUCX_Ubv-VZpLUMwiHYw                       Wing China   
9   LoJ8AmYraU7q3o8S3HY-4w               Bennie's Pizza Pub   
10  gfQmLdpBzhs_y1sQhIzjBQ         Casa Maria Mexican Grill   
11  VVC2ZRQlz8JtK9LmgjQoIw             Sushi Thai by Samran   
12  d-KScItGomWp3EqbXrfZ6w                      Chick-fil-A   
13  eVPJcdu8vlsjrWFC3VHiMA                    White Cottage   
14  pEvFMovlRwKvG8dqdnZv2A           AJs Smashed and Sm

In [ ]:
###geographical data visualization

import folium
from folium.plugins import MarkerCluster

In [ ]:
location = [result_df['latitude'].mean(), result_df['longitude'].mean()]

# Create a map centered on Illinois
m = folium.Map(location=location, zoom_start=7, width='70%',height='50%')  # Coordinates are centered on Illinois

# Add a marker cluster to handle multiple markers in close proximity
marker_cluster = MarkerCluster().add_to(m)

# Add each restaurant as a marker on the map
for idx, row in result_df.iterrows():
    folium.Marker(
        location=[row['latitude'], row['longitude']],
        popup=f"{row['name']} - {row['stars']} stars - {row['review_count']} reviews\n"
              f"City: {row['city']} - High Rated Restaurants in City: {row['high_rated_restaurant_count']}",
        tooltip=row['name']
    ).add_to(marker_cluster)

# Display the map
m

In [ ]:

from sqlalchemy import text
from sqlalchemy import create_engine, text
###2. Cities with High Percentage of Highly Rated Restaurants in Illinois
query = """
SELECT
    b.city,
    COUNT(CASE WHEN b.stars >= 4 THEN 1 END) * 100.0 / COUNT(b.business_id) AS high_rating_percentage,
    COUNT(b.business_id) AS restaurant_count
FROM business AS b
WHERE b.state = 'IL' AND b.categories LIKE '%Restaurant%'
GROUP BY b.city
HAVING COUNT(b.business_id) > 5
ORDER BY high_rating_percentage DESC;
"""

try:
    with engine.connect() as connection:
        result_df = pd.read_sql_query(text(query), con=connection)
        display(result_df)
except Exception as e:
    print(f"An error occurred: {e}")

,city,high_rating_percentage,restaurant_count
0,East Saint Louis,71.428571,7
1,Waterloo,53.125000,32
2,Godfrey,50.000000,16
3,Granite City,48.000000,50
4,Lebanon,47.368421,19
5,Millstadt,46.153846,13
6,Wood River,41.379310,29
7,Alton,40.476190,84
8,East Alton,36.363636,11
9,O'Fallon,36.065574,61


In [ ]:
###3 find the city in IL with the most high rated restaurants
table_name = "business"  # Replace with the table name you want to verify
query = f"SELECT TOP 20 *, COUNT(*) OVER (PARTITION BY city) AS high_rated_restaurant_count FROM {table_name} WHERE stars >= 3.5  AND [state] = 'IL'  AND categories LIKE '%Restaurants%' AND is_open = 1 ORDER BY high_rated_restaurant_count DESC ;"

# Execute the query and fetch results in a DataFrame
try:
    with engine.connect() as connection:
        # Execute the query and load the result into a DataFrame
        result_df = pd.read_sql_query(text(query), con=connection)
        display(result_df)
except Exception as e:
    print(f"An error occurred: {e}")



,business_id,name,address,city,state,postal_code,latitude,longitude,stars,review_count,...,attributes_DietaryRestrictions,hours_Monday,hours_Tuesday,hours_Wednesday,hours_Thursday,hours_Friday,hours_Saturday,hours_Sunday,pk_business_id_1,high_rated_restaurant_count
0,RQPDg_bnyjPrk1xiWf3lpA,Uncle Nick's Deli,1140 S Illinois St,Belleville,IL,62226,38,-90,4,29,...,0,0,0,0,0,0,0,0,139764,34
1,2UtP4mly2LFZpXtqiwb24g,The Weingarten,1780 E State Rt 15,Belleville,IL,62221,38,-89,4,30,...,0,5:0-23:0,5:0-23:0,5:0-23:0,5:0-23:0,5:0-23:0,5:0-23:0,6:0-23:0,122353,34
2,Gt6NclvT89Je0_VHI1eNrQ,Sala Thai Cuisine,1229 Lebanon Ave,Belleville,IL,62221,38,-89,4,71,...,0,7:0-22:0,7:0-22:0,7:0-22:0,7:0-22:0,7:0-22:0,9:0-22:0,7:0-22:0,119699,34
3,omKVwCFLQYs_ySD1bPk6iw,Eckert's Country Store & Farms,951 S Green Mount Rd,Belleville,IL,62220,38,-89,4,401,...,0,9:30-20:0,9:30-20:0,9:30-20:0,9:30-20:0,0,9:0-15:0,9:0-15:0,114468,34
4,uT33RCQUJioJnFEh1C1u7g,Main Street Cafe,1601 W Main St,Belleville,IL,62220,38,-90,4,82,...,0,9:0-18:0,9:0-18:0,9:0-18:0,9:0-18:0,9:0-18:0,10:0-17:0,13:0-17:0,108094,34
5,P9kDKqqjdFjjpWIAaxNyRg,Papa Vito's Belleville,318 E Washington,Belleville,IL,62220,38,-89,4,107,...,0,8:0-17:0,8:0-17:0,8:0-17:0,8:0-17:0,8:0-17:0,8:0-12:0,0,113192,34
6,S5SYHcyBG9HuaprVO_6jFQ,Sip & Chew Express,124 W Main St,Belleville,IL,62220,38,-89,4,18,...,0,10:0-19:30,10:0-19:30,10:0-19:30,10:0-19:30,10:0-19:30,10:0-18:30,10:0-17:0,101655,34
7,-VvhvE-Qs0kjoGidq7XYDQ,Cured And Cultured,6401 W Main St,Belleville,IL,62223,38,-90,4,19,...,0,0:0-0:0,5:0-18:30,5:0-15:30,5:0-18:30,5:0-15:30,6:0-11:0,0,101767,34
8,fVBUCX_Ubv-VZpLUMwiHYw,Wing China,9 Bellevue Park Plz,Belleville,IL,62226,38,-90,4,8,...,0,0:0-0:0,7:30-17:30,7:30-17:30,7:30-17:30,7:30-17:30,7:30-16:0,0,95770,34
9,LoJ8AmYraU7q3o8S3HY-4w,Bennie's Pizza Pub,113 E Main St,Belleville,IL,62220,38,-89,4,47,...,0,0:0-0:0,9:30-19:30,9:30-19:30,9:30-19:30,9:30-19:30,9:30-18:30,0,103260,34


In [ ]:
# 4: Calculate review-to-restaurant ratio for each city in Illinois
query = """
WITH city_reviews AS (
    SELECT
        city,
        SUM(review_count) AS total_reviews,
        COUNT(business_id) AS restaurant_count,
        CAST(SUM(review_count) AS FLOAT) / COUNT(business_id) AS review_to_restaurant_ratio
    FROM business
    WHERE state = 'IL' AND categories LIKE '%Restaurant%'
    GROUP BY city
)

-- Step 2: Retrieve data for Belleville and the city with the second highest review count
SELECT city, total_reviews, restaurant_count, review_to_restaurant_ratio
FROM city_reviews
WHERE city IN (
    SELECT TOP 2 city
    FROM city_reviews
    ORDER BY total_reviews DESC
)
ORDER BY review_to_restaurant_ratio DESC;
"""

try:
    with engine.connect() as connection:
        # Execute the query and load the result into a DataFrame
        result_df = pd.read_sql_query(text(query), con=connection)
        display(result_df)

        # Interpretation of the results to find the best city for a new restaurant
        if len(result_df) == 2:
            city_1, city_2 = result_df['city'].iloc[0], result_df['city'].iloc[1]
            ratio_1, ratio_2 = result_df['review_to_restaurant_ratio'].iloc[0], result_df['review_to_restaurant_ratio'].iloc[1]
            if ratio_1 > ratio_2:
                print(f"{city_1} has a much higher review-to-restaurant ratio than {city_2}, suggesting a larger customer base in {city_1}, making it a better choice for a new restaurant.")
            else:
                print(f"{city_2} has a much higher review-to-restaurant ratio than {city_1}, suggesting a larger customer base in {city_2}, making it a better choice for a new restaurant.")

except Exception as e:
    print(f"An error occurred: {e}")

,city,total_reviews,restaurant_count,review_to_restaurant_ratio
0,Edwardsville,6318,128,49.359375
1,Belleville,5560,146,38.082192


Edwardsville has a much higher review-to-restaurant ratio than Belleville, suggesting a larger customer base in Edwardsville, making it a better choice for a new restaurant.


In [ ]:
###5. Find the top five cities in Illinois with the most restaurants

table_name = "business"

query = f"""
SELECT TOP 5 city, COUNT(*) AS restaurant_count
FROM {table_name}
WHERE [state] = 'IL' AND categories LIKE '%Restaurant%' AND is_open = 1
GROUP BY city
ORDER BY restaurant_count DESC;
"""

# Execute the query and fetch results in a DataFrame
try:
    with engine.connect() as connection:
        # Execute the query and load the result into a DataFrame
        result_df = pd.read_sql_query(text(query), con=connection)
        print(result_df)
except Exception as e:
    print(f"An error occurred: {e}")

               city  restaurant_count
0        Belleville               100
1      Edwardsville                89
2             Alton                63
3      Collinsville                59
4  Fairview Heights                53


In [ ]:
###6. Calculate the average number of reviews per restaurant in the city

query = """
SELECT AVG(review_count) AS avg_reviews
FROM business
WHERE business.city = 'Belleville' AND categories LIKE '%Restaurant%';
"""

try:
    with engine.connect() as connection:
        result_df = pd.read_sql_query(text(query), con=connection)
        print(result_df)
except Exception as e:
    print(f"An error occurred: {e}")




   avg_reviews
0           38


The average number of reviews in Belleville per restaurant is **38**

In [ ]:
##7. Find top-rated restaurants in the city

query = """
SELECT TOP 10
    name,
    stars,
    review_count,
    LTRIM(RTRIM(REPLACE(REPLACE(REPLACE(categories, 'Restaurants, ', ''), ', Restaurants', ''), 'Restaurants', ''))) AS cuisine
FROM business
WHERE business.city = 'Belleville' AND categories LIKE '%Restaurant%'
ORDER BY stars DESC, review_count DESC;
"""

try:
    with engine.connect() as connection:
        result_df = pd.read_sql_query(text(query), con=connection)
        display(result_df)
except Exception as e:
    print(f"An error occurred: {e}")




,name,stars,review_count,cuisine
0,C And C Food For Your Soul,5,12,"Southern, American (Traditional), Soul Food"
1,Nick and Vito's,5,10,"Food, Butcher, Delis"
2,Lucinda's Grill and Cafe,5,5,Cafes
3,BEAST Craft BBQ,4,583,Barbeque
4,Eckert's Country Store & Farms,4,401,"Comfort Food, Arts & Entertainment, Farms, Spe..."
5,Casa Maria Mexican Grill,4,161,"Tacos, Cocktail Bars, Bars, Nightlife, Mexican..."
6,Tavern On Main,4,134,"Italian, Bars, Nightlife"
7,Papa Vito's Belleville,4,107,Pizza
8,Saigon,4,95,"Specialty Food, Imported Food, Chinese, Vietna..."
9,The Wine Tap,4,93,"Bars, Wine Bars, Nightlife, Italian"


In [ ]:
##8. Find the most popular cuisines in a specific city

query = """
SELECT TOP 10
    LTRIM(RTRIM(REPLACE(REPLACE(REPLACE(categories, 'Restaurants, ', ''), ', Restaurants', ''), 'Restaurants', ''))) AS category_cleaned,
    COUNT(business_id) AS num_restaurants
FROM business
WHERE business.city = 'Belleville' AND categories LIKE '%Restaurant%'
GROUP BY LTRIM(RTRIM(REPLACE(REPLACE(REPLACE(categories, 'Restaurants, ', ''), ', Restaurants', ''), 'Restaurants', '')))
ORDER BY num_restaurants DESC;
"""

try:
    with engine.connect() as connection:
        # Execute the query and load the result into a DataFrame
        result_df = pd.read_sql_query(text(query), con=connection)
        display(result_df)
except Exception as e:
    print(f"An error occurred: {e}")






,category_cleaned,num_restaurants
0,Chinese,5
1,Barbeque,4
2,American (Traditional),3
3,"Fast Food, Burgers",3
4,Mexican,3
5,Pizza,3
6,"Burgers, Hot Dogs, Fast Food",2
7,Cafes,2
8,"Chicken Wings, Fast Food",2
9,"Food, Ice Cream & Frozen Yogurt, Fast Food, Bu...",2


The most popular restaurants in Belleville are Chinese, Barbeque and American (Traditional)


In [ ]:
###9. Analyze customer reviews to identify common reviews and conduct sentiment analysis
from textblob import TextBlob
from sqlalchemy import create_engine, text
import pandas as pd

# Step 1: Retrieve top-rated restaurants in Belleville
top_restaurants_query = """
SELECT TOP 10
    business_id,
    name,
    stars,
    review_count,
    LTRIM(RTRIM(REPLACE(REPLACE(REPLACE(categories, 'Restaurants, ', ''), ', Restaurants', ''), 'Restaurants', ''))) AS cuisine
FROM business
WHERE business.city = 'Belleville' AND categories LIKE '%Restaurant%'
ORDER BY stars DESC, review_count DESC;
"""

# Step 2: Fetch the reviews for these top-rated restaurants and perform sentiment analysis
try:
    with engine.connect() as connection:
        # Execute the query to get top-rated restaurants
        top_restaurants_df = pd.read_sql_query(text(top_restaurants_query), con=connection)

        # Extract the business IDs of these restaurants
        business_ids = tuple(top_restaurants_df['business_id'].tolist())

        # Query to get reviews for the top restaurants
        reviews_query = f"""
        SELECT text, business_id
        FROM review
        WHERE business_id IN {business_ids}
        """

        # Execute the reviews query and load the results into a DataFrame
        reviews_df = pd.read_sql_query(text(reviews_query), con=connection)

        # Perform sentiment analysis on each review
        def get_sentiment_score(text):
            analysis = TextBlob(text)
            return analysis.sentiment.polarity  # Returns a score between -1 and 1

        # Apply sentiment analysis to each review
        reviews_df['sentiment_score'] = reviews_df['text'].apply(get_sentiment_score)

        # Step 3: Calculate the average sentiment score for each restaurant
        avg_sentiment_df = reviews_df.groupby('business_id')['sentiment_score'].mean().reset_index()
        avg_sentiment_df.rename(columns={'sentiment_score': 'average_sentiment_score'}, inplace=True)

        # Merge the sentiment scores with the top-rated restaurants DataFrame
        result_df = pd.merge(top_restaurants_df, avg_sentiment_df, on='business_id')

        # Step 4: Sort by average sentiment score in descending order
        result_df = result_df.sort_values(by='average_sentiment_score', ascending=False)

        # Display the final result
        display(result_df)

except Exception as e:
    print(f"An error occurred: {e}")

###	•	1 indicates the most positive sentiment.
### •	0 indicates a neutral sentiment.
### •	-1 indicates the most negative sentiment.



,business_id,name,stars,review_count,cuisine,average_sentiment_score
1,0L5RXMLlh1AnzC-nu4oYQA,Nick and Vito's,5,10,"Food, Butcher, Delis",0.435970
9,JRoY9Mx2LghIki2Uo1SOJA,The Wine Tap,4,93,"Bars, Wine Bars, Nightlife, Italian",0.360756
3,nwN92Uje-xIKE5voPTTvBQ,BEAST Craft BBQ,4,583,Barbeque,0.358404
5,gfQmLdpBzhs_y1sQhIzjBQ,Casa Maria Mexican Grill,4,161,"Tacos, Cocktail Bars, Bars, Nightlife, Mexican...",0.342429
6,S9tdibDgtvdWLiyINyi4nw,Tavern On Main,4,134,"Italian, Bars, Nightlife",0.305227
2,kxs_y4UlQl3p1ByNPB4H7A,Lucinda's Grill and Cafe,5,5,Cafes,0.287139
4,omKVwCFLQYs_ySD1bPk6iw,Eckert's Country Store & Farms,4,401,"Comfort Food, Arts & Entertainment, Farms, Spe...",0.278488
7,P9kDKqqjdFjjpWIAaxNyRg,Papa Vito's Belleville,4,107,Pizza,0.266993
0,hhV_G25PpQ77qT5XdTd00g,C And C Food For Your Soul,5,12,"Southern, American (Traditional), Soul Food",0.266927
8,7P6-It26918v6JDWcYlKSw,Saigon,4,95,"Specialty Food, Imported Food, Chinese, Vietna...",0.263494


In [ ]:
##10.Find the most popular price range among top-rated restaurants


query = """
SELECT attributes_RestaurantsPriceRange2 AS price_range, COUNT(business_id) AS num_restaurants
FROM business
WHERE business.city = 'Belleville' AND categories LIKE '%Restaurant%' AND stars >= 3.5
GROUP BY attributes_RestaurantsPriceRange2
ORDER BY num_restaurants DESC;
"""

try:
    with engine.connect() as connection:
        result_df = pd.read_sql_query(text(query), con=connection)
        display(result_df)
except Exception as e:
    print(f"An error occurred: {e}")



,price_range,num_restaurants
0,0,17
1,1,15
2,2,15
3,3,2
4,4,1


**what is price range?**

The price range in this context refers to a rating that indicates the relative cost of a restaurant.

In many datasets, including Yelp, price ranges are often represented by a numerical scale, with each number roughly corresponding to a level of expense.
Typically, they are represented as follows:

 	•	1: Low price range (e.g., inexpensive, budget-friendly).
 	•	2: Moderate price range (e.g., average pricing, mid-range).
 	•	3: High price range (e.g., upscale, expensive).
 	•	4: Very high price range (e.g., luxury, premium dining).

 **most of the restaurants are budget friendly, we should keep our restaurant in the low price range**

Based on the above analysis, we decided to open a **Chinese** in

---

Belleville